In [1]:
import tensorflow as tf
import os
os.environ["TF_USE_LEGACY_KERAS"]="1"
import keras
import tensorflow_model_optimization as tfmot
from sklearn.metrics.cluster import entropy
from tf_keras import layers,losses,optimizers,Sequential
from tf_keras.models import Model
from tf_keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D,Input
from tf_keras.applications import VGG16
from tf_keras import datasets,models
import importlib

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [66]:
from importing_dataset import load_dataset
train_examples, validation_examples, num_examples, num_classes, class_names = load_dataset('horses_or_humans', 70)

INFO:absl:Load dataset info from C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0
INFO:absl:Creating a tf.data.Dataset reading 2 files located in folders: C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0.
INFO:absl:Creating a tf.data.Dataset reading 1 files located in folders: C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0.
INFO:absl:Constructing tf.data.Dataset horses_or_humans for split ('train[:70%]', 'train[70%:]'), from C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0


In [4]:
from reformatting import reformat_image
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
train_batches = train_examples.cache().shuffle(num_examples//4).map(reformat_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(reformat_image).batch(BATCH_SIZE).prefetch(1)
train_batches

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [81]:
import Entropy_Keras_Surgeon
importlib.reload(Entropy_Keras_Surgeon)
from Entropy_Keras_Surgeon import EntropyPruningSurgeon
testt_model = models.load_model("model_experiments/custom_vgg16.keras")
pruning = EntropyPruningSurgeon(model=testt_model, threshold=0.3)
pruningmodel = pruning.run()
pruningmodel.summary()

Deleting 63/64 channels from layer: block1_conv1
Deleting 63/64 channels from layer: block1_conv2
Deleting 127/128 channels from layer: block2_conv1
Deleting 127/128 channels from layer: block2_conv2
Deleting 255/256 channels from layer: block3_conv1
Deleting 255/256 channels from layer: block3_conv2
Deleting 255/256 channels from layer: block3_conv3
Deleting 511/512 channels from layer: block4_conv1
Deleting 511/512 channels from layer: block4_conv2
Deleting 511/512 channels from layer: block4_conv3
Deleting 511/512 channels from layer: block5_conv1
Deleting 511/512 channels from layer: block5_conv2
Deleting 511/512 channels from layer: block5_conv3
Model: "model_101"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_49 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 input_40 (InputLayer)       multiple                  0  

In [44]:
# Test the model
pruningmodel.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer = optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])


In [45]:
pruningmodel.evaluate(validation_batches)

10/10 [==============================] - 41s 4s/step - loss: 2.6552 - accuracy: 0.8669


[2.6551754474639893, 0.8668830990791321]

In [68]:
import tempfile
def prune_model_entropy(model, epochs,threshold, train_data=train_batches, val_data=validation_batches):

  # Create a tensorboard logfile
  logdir = tempfile.mkdtemp()
  # The end_step is the total number of iterations required for the training data which is basically the entire epochs over the length of the training data
  end_step = int(len(train_data) * epochs * 0.5)

  learning_rate_fn = optimizers.schedules.PolynomialDecay(
    0.001,
    1000,
    0.0001,
    power=0.5)

  # Model for pruning
  entropy_pruning_instance = EntropyPruningSurgeon(model=model, threshold=threshold)
  model_for_pruning = entropy_pruning_instance.run()

  # Recompile
  model_for_pruning.compile(optimizer= optimizers.Adam(),
                            loss= losses.SparseCategoricalCrossentropy(),
                            metrics=["accuracy"])

  # Fit the model
  model_for_pruning.fit(train_data,
                      validation_data=val_data,
                      batch_size=BATCH_SIZE,
                      epochs=epochs)

  # Evaluate the model
  score = model_for_pruning.evaluate(val_data, verbose=0)
  metric_dict = {
      "entropy" : entropy,
      "val_loss" : np.round(score[0], 4),
      "val_accuracy" : np.round(score[1] * 100, 4)
  }
  return logdir, metric_dict, model_for_pruning

In [69]:
# Choose the best entropy level
k_entropies = np.arange(0.019, 0.029, 0.001)
metric_list = []

for k in k_entropies:
  # Load in the best saved model
  model_1 = models.load_model("model_experiments/model_horses_or_humans.keras")
  logdir, metrics, pruned_model = prune_model_entropy(model=model_1,
            threshold=k,
            epochs=20)
  val_loss, val_accuracy  = metrics["val_loss"], metrics["val_accuracy"]
  metric_list.append(metrics)
  print(f"Entropy : {k} \tValidation Loss: {val_loss}, \tValidation Accuracy: {val_accuracy}")

# Create a dataframe of the values obtained
df = pd.DataFrame(metric_list)

# Save the dataframe to a csv file
df.to_csv("entropy_pruning_results.csv")

Epoch 1/20
 9/23 [==========>...................] - ETA: 1:08 - loss: 435.7670 - accuracy: 0.7535

KeyboardInterrupt: 

In [196]:
model_for_export = tfmot.sparsity.keras.strip_pruning(pruned_model)

_, pruned_keras_file = tempfile.mkstemp('.h5')
models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

C:\Users\neyen\AppData\Local\Temp\ipykernel_71452\369930696.py:4: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


Saved pruned Keras model to: C:\Users\neyen\AppData\Local\Temp\tmpjh1lij65.h5
